In [1]:
from __future__ import division

import numpy as np
import inspect
import functools
import matplotlib.pyplot as plt

from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
import enterprise.signals.signal_base as base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from tests.enterprise_test_data import datadir
from enterprise.signals import white_signals
from enterprise.signals import gp_signals



% matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
psr = Pulsar(datadir+'/B1855+09_NANOGrav_11yv0.gls.par', datadir+'/B1855+09_NANOGrav_11yv0.tim')

## Updated fourier design matrix and test function that has `Parameter`s

In [3]:
def createfourierdesignmatrix_env(toas, log10_Amp=-7, log10_Q=np.log10(300), t0=4783728350.5632496, 
                                  nmodes=30, Tspan=None, logf=False, fmin=None, 
                                  fmax=None):


    # get base fourier design matrix and frequencies
    F, Ffreqs = utils.createfourierdesignmatrix_red(
        toas, nmodes=nmodes, Tspan=Tspan, logf=logf,
        fmin=fmin, fmax=fmax)

    # compute gaussian envelope
    A = 10**log10_Amp
    Q = 10**log10_Q * 86400
    env = A * np.exp(-(toas-t0)**2/2/Q**2)
    return F, Ffreqs

In [16]:
pl = base.Function(utils.powerlaw, log10_A=parameter.Uniform(-18,-12), 
              gamma=parameter.Uniform(0, 7))
basis_env = base.Function(createfourierdesignmatrix_env)
basis_red = base.Function(utils.createfourierdesignmatrix_red)
basis_red2 = base.Function(utils.createfourierdesignmatrix_red, nmodes=50)
selection = Selection(selections.by_backend)

rn = gp_signals.BasisGP(pl, basis_env, name='env')
rn2 = gp_signals.BasisGP(pl, basis_red)
rn3 = gp_signals.BasisGP(pl, basis_red2, name='red2')
wn = white_signals.MeasurementNoise()
m = (rn + wn + rn2 + rn3)(psr)

In [17]:
m.params

["B1855+09_efac":Uniform(0.5,1.5),
 "B1855+09_env_gamma":Uniform(0,7),
 "B1855+09_env_log10_A":Uniform(-18,-12),
 "B1855+09_gamma":Uniform(0,7),
 "B1855+09_log10_A":Uniform(-18,-12),
 "B1855+09_red2_gamma":Uniform(0,7),
 "B1855+09_red2_log10_A":Uniform(-18,-12)]

In [18]:
params = {'B1855+09_log10_A':-14, 'B1855+09_gamma':3.5, 'B1855+09_env_log10_Amp':-7}
#Fmat = m.get_basis(params)
phi = m.get_phi(params)

In [20]:
phi.shape

(100,)

## Test out the new method on a Fourier Basis signal with basis parameters

In [8]:
# set up powerlaw spectrum
pl = Function(utils.powerlaw, log10_A=parameter.Uniform(-18,-12), 
              gamma=parameter.Uniform(0, 7))

# set up Gaussian envelope basis function
log10_Amp = parameter.Uniform(-10, -5)
log10_Q = parameter.Uniform(np.log10(30), np.log10(3000))
t0 = parameter.Uniform(psr.toas.min(), psr.toas.max())
fourier_env = Function(createfourierdesignmatrix_env, 
                       t0=t0, log10_Amp=log10_Amp, log10_Q=log10_Q, 
                       nmodes=50)

# make signal
gp = BasisGP(pl, fourier_env, name='env')
gpm = gp(psr)

NameError: name 'Function' is not defined

In [ ]:
gpm.params

In [ ]:
# parameters
params = {'B1855+09_env_log10_A': -15,
          'B1855+09_env_gamma': 4.33,
          'B1855+09_env_log10_Amp': -8,
          'B1855+09_env_log10_Q': np.log10(300),
          'B1855+09_env_t0': 4783728350.5632496}

F = gpm.get_basis()
phi = gpm.get_phi(params)

In [ ]:
plt.plot(psr.toas, F[:,15])

## Test out the new method with ECORR basis

In [ ]:
# slightly modified quantization matrix
def create_quantization_matrix(toas, dt=1, nmin=2):
    """Create quantization matrix mapping TOAs to observing epochs."""
    isort = np.argsort(toas)

    bucket_ref = [toas[isort[0]]]
    bucket_ind = [[isort[0]]]

    for i in isort[1:]:
        if toas[i] - bucket_ref[-1] < dt:
            bucket_ind[-1].append(i)
        else:
            bucket_ref.append(toas[i])
            bucket_ind.append([i])

    # find only epochs with more than 1 TOA
    bucket_ind2 = [ind for ind in bucket_ind if len(ind) >= nmin]

    U = np.zeros((len(toas),len(bucket_ind2)),'d')
    for i,l in enumerate(bucket_ind2):
        U[l,i] = 1
        
    weights = np.ones(U.shape[1])

    return U, weights

# ecorr prior function
def ecorr_basis_prior(weights, log10_ecorr=-8):
    return weights * 10**(2*log10_ecorr)

In [ ]:
basisFunction = Function(create_quantization_matrix)
priorFunction = Function(ecorr_basis_prior, log10_ecorr=parameter.Uniform(-10, -5))
selection = Selection(selections.by_backend)

ec = BasisGP(priorFunction, basisFunction, selection=selection)
ecm = ec(psr)

In [ ]:
ecm.params

In [ ]:
params = {'B1855+09_430_PUPPI_log10_ecorr': -7.2, 
          'B1855+09_L-wide_ASP_log10_ecorr': -8.2, 
          'B1855+09_430_ASP_log10_ecorr': -6.4, 
          'B1855+09_L-wide_PUPPI_log10_ecorr': -7}

U = ecm.get_basis(params=params)
phi = ecm.get_phi(params=params)

In [ ]:
phi